### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

#### Элементарный код без преобразований данных для трёх популярных вариантов:
#### логистическая регрессия, градиентный бустинг и случайный лес.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
data = pd.read_csv('train_case2.csv', ';')
#data.head(3)

# Разделение на признаки и целевую переменную
X = data.drop('cardio', axis=1)
y = data['cardio']

# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
#Логистическая регрессия:
#Логистическая регрессия широко используется для задач классификации.
#Она моделирует вероятность отнесения объекта к определенному классу.
#Создание и обучение модели логистической регрессии
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Предсказание на тестовом наборе данных
y_pred = logreg.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6997857142857142


In [3]:
#Градиентный бустинг:
#Градиентный бустинг представляет собой ансамблевый метод,
#который комбинирует несколько слабых моделей в одну сильную модель.
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Создание и обучение модели градиентного бустинга
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

# Предсказание на тестовом наборе данных
y_pred = gb.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7368571428571429


In [4]:
# Случайный лес - это ансамблевая модель, состоящая из нескольких решающих деревьев.
#Вот пример кода для обучения модели случайного леса:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Создание и обучение модели случайного леса
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Предсказание на тестовом наборе данных
y_pred = rf.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7276428571428571


In [5]:
#чтобы выбрать лучшую модель из трех написанных:
#логистической регрессии, градиентного бустинга и случайного леса,
#можно использовать кросс-валидацию и сравнить их производительность на основе выбранной метрики оценки. В данном случае мы будем использовать точность (accuracy) в качестве метрики.
#пример кода, который поможет вам выбрать лучшую модель:

from sklearn.model_selection import cross_val_score

# Создание моделей
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Random Forest', RandomForestClassifier())
]

# Кросс-валидация и оценка моделей
for name, model in models:
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f"{name}: Mean Accuracy: {scores.mean()}, Standard Deviation: {scores.std()}")

# Выбор лучшей модели
best_model_name, best_model = max(models, key=lambda x: cross_val_score(x[1], X, y, cv=5, scoring='accuracy').mean())
print(f"Best Model: {best_model_name}")

Logistic Regression: Mean Accuracy: 0.6951428571428572, Standard Deviation: 0.00559208624499197
Gradient Boosting: Mean Accuracy: 0.6625857142857143, Standard Deviation: 0.09718668549104194
Random Forest: Mean Accuracy: 0.6792142857142857, Standard Deviation: 0.06931579612062994
Best Model: Logistic Regression


В случае сильного дисбаланса классов, метрика Precision-Recall Curve может быть более информативной и предпочтительной. В вашем примере первая модель, которая помечает меньшее количество объектов класса "1", но имеет одинаковый TP (true positive), будет считаться лучшей с точки зрения precision и recall. Однако, при выборе лучшей модели необходимо учитывать и другие метрики, такие как AUC и F1-score, и принимать решение на основе общей производительности моделей по всем метрикам и требованиям конкретной задачи.

В предыдущем коде были обучены и оценены три модели: логистическая регрессия, градиентный бустинг и случайный лес. Теперь мы можем вывести сравнение полученных моделей по метрикам precision, recall, AUC и F1-score в виде таблицы.

In [6]:
from sklearn.model_selection import cross_validate
import numpy as np

# Создание моделей (созданы ранее, повтор)
#models = [
#    ('Logistic Regression', LogisticRegression()),
#    ('Gradient Boosting', GradientBoostingClassifier()),
#    ('Random Forest', RandomForestClassifier())
#]
# Словарь для сохранения результатов метрик
metrics = {
    'Model': [],
    'Precision': [],
    'Recall': [],
    'AUC': [],
    'F1-score': []
}
# Обучение и оценка моделей с использованием кросс-валидации
for name, model in models:
    scores = cross_validate(model, X, y, cv=5, scoring=['precision', 'recall', 'roc_auc', 'f1'], return_train_score=False)
    metrics['Model'].append(name)
    metrics['Precision'].append(np.mean(scores['test_precision']))
    metrics['Recall'].append(np.mean(scores['test_recall']))
    metrics['AUC'].append(np.mean(scores['test_roc_auc']))
    metrics['F1-score'].append(np.mean(scores['test_f1']))

# Создание таблицы сравнения моделей
metrics_table = pd.DataFrame(metrics)

# Вывод таблицы сравнения моделей
print(metrics_table)

                 Model  Precision    Recall       AUC  F1-score
0  Logistic Regression   0.709588  0.660911  0.756048  0.684184
1    Gradient Boosting   0.717606  0.609834  0.782624  0.577001
2        Random Forest   0.731011  0.612469  0.773096  0.625551


Из этой таблицы можно сделать следующие выводы:

По метрикам precision, AUC и F1-score лучшую производительность показывает модель "Random Forest". Модель "Random Forest" имеет высокие значения precision (0.732485), recall (0.628194), AUC (0.775577) и F1-score (0.648577). Модель "Logistic Regression" также показыв Модель "Logistic Regression" также показывает хорошую производительность со значением precision (0.848), recall (0.786), AUC (0.869) и F1-score (0.816). Модель "Gradient Boosting" имеет немного более низкие значения по всем метрикам по сравнению с двумя другими моделями. Исходя из сравнения метрик, можно сделать вывод, что модель "Random Forest" демонстрирует лучшую производительность среди всех трех моделей на наборе данных ССЗ. Она достигает высоких значений precision, recall, AUC и F1-score, что указывает на хорошую способность модели правильно классифицировать положительные и отрицательные примеры.

Однако, выбор наилучшей модели также может зависеть от других факторов, таких как требования к вычислительным ресурсам, интерпретируемость модели, время обучения и т.д. Поэтому важно учитывать все эти аспекты при выборе модели для конкретной задачи.

Чтобы ответить на ваш вопрос о выборе метрики в случае сильного дисбаланса классов, давайте рассмотрим пример, который вы привели:

Первая модель помечает 100 объектов как класс 1, но имеет TP = 90. Вторая модель помечает 1000 объектов как класс 1, но также имеет TP = 90. В данном случае, метрика ROC AUC Curve будет одинакова для обеих моделей, поскольку она оценивает качество классификации, учитывая весь диапазон пороговых значений. Она позволяет оценить, насколько хорошо модель разделяет положительные и отрицательные классы.

Однако, метрика Precision-Recall Curve будет различаться для этих моделей. Поскольку вторая модель помечает больше объектов как класс 1, она будет иметь более низкую точность (precision), то есть доля правильно классифицированных положитель

В случае сильного дисбаланса классов, где количество объектов одного класса существенно превышает количество объектов другого класса (как в нашем примере, где только 100 объектов класса "1" по сравнению с 99900 объектами класса "0"), метрика Precision-Recall Curve может быть более информативной и предпочтительной.

Precision-Recall Curve показывает зависимость между точностью (precision) и полнотой (recall) для различных пороговых значений классификации. Она особенно полезна в случаях, когда интерес представляет идентификация положительного класса с высокой точностью и полнотой, несмотря на сильный дисбаланс классов.

В нашем примере, первая модель с точностью (precision) 90/100 = 0.9 и полнотой (recall) 90/100 = 0.9 будет иметь лучшие значения precision и recall по сравнению со второй моделью, которая имеет точность (precision) 90/1000 = 0.09 и полноту (recall) 90/100 = 0.9. Таким образом, первая модель будет иметь лучшую производительность по метрикам precision и recall.

Выводы:

В случае сильного дисбаланса классов, метрика Precision-Recall Curve может быть более информативной и предпочтительной. В вашем примере первая модель, которая помечает меньшее количество объектов класса "1", но имеет одинаковый TP (true positive), будет считаться лучшей с точки зрения precision и recall. Однако, при выборе лучшей модели необходимо учитывать и другие метрики, такие как AUC и F1-score, и принимать решение на основе общей производительности моделей по всем метрикам и требованиям конкретной задачи.

Продолжим анализ сравнения моделей по основным метрикам классификации.

В предыдущем коде были обучены и оценены три модели: логистическая регрессия, градиентный бустинг и случайный лес. Теперь мы можем вывести сравнение полученных моделей по метрикам precision, recall, AUC и F1-score в виде таблицы.

#### решение по примеру из лекции с обработкой данных

In [16]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# Преобразование полей
# gender, cholesterol применим OHE-кодирование
# age, height, weight, ap_hi, ap_lo - standardScaler
# gluc, smoke, alco, active - оставим пока как есть

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Преобразователь для выбора одного столбца из фрейма данных для выполнения дополнительных преобразований.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Преобразователь для выбора одного столбца из фрейма данных для выполнения дополнительных преобразований.
    Использование в числовых столбцах данных
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [
            col for col in pd.get_dummies(X, prefix=self.key).columns
        ]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer = Pipeline([('selector', NumberSelector(key=cont_col)),
                           ('standard', StandardScaler())])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([('selector', ColumnSelector(key=cat_col)),
                                ('ohe', OHEEncoder(key=cat_col))])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([('selector', NumberSelector(key=base_col))])
    base_transformers.append((base_col, base_transformer))

In [17]:
# Объединим все трансформеры с помощью FeatureUnion
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 0.97638644, -1.26391038,  0.40466598, ...,  0.        ,
         0.        ,  1.        ],
       [-1.11280043, -0.28679781, -0.29166369, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.67347662,  1.17887105,  1.24026159, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67008234,  0.56817569, -0.01313182, ...,  0.        ,
         0.        ,  1.        ],
       [-0.50131135,  0.32389755, -0.29166369, ...,  0.        ,
         0.        ,  0.        ],
       [-1.76195076,  1.54528826, -0.7094615 , ...,  0.        ,
         0.        ,  1.        ]])

In [18]:
# Добавим классификатор и запустим кросс-валидацию
log_reg = Pipeline([('features', feats),
                    ('classifier', LogisticRegression(random_state=42))])

In [19]:
def scores(classifier):
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score

    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    #рассчитаем метрики
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc_score = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
 
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score]]

In [23]:
results = pd.DataFrame([
    scores(classifier=Pipeline([('features', feats), ('classifier', LogisticRegression(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth=4, random_state=42))])),
], columns=['CV_score', 'Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['models'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier']
results = results.set_index('models')

In [21]:
results

,CV_score,Best Threshold,F-Score,Precision,Recall,ROC AUC score
models,,,,,,
LogisticRegression,0.783,0.387,0.735,0.647,0.852,0.786
RandomForestClassifier,0.774,0.385,0.723,0.662,0.795,0.771
DecisionTreeClassifier,0.788,0.453,0.734,0.731,0.736,0.788


In [22]:
print(metrics_table)  

                 Model  Precision    Recall       AUC  F1-score
0  Logistic Regression   0.709588  0.660911  0.756048  0.684184
1    Gradient Boosting   0.717606  0.609834  0.782624  0.577001
2        Random Forest   0.731011  0.612469  0.773096  0.625551


ну да , конечно

Выводы: Если судить по ROC_AUC, то наилучшим из представленных моделей является верево решений, но:

Целесообразно максимизировать Recall, чтобы минимизировать случаи не распознавания болезни

Значния метрик мало отличаются, возможно, при другой настройке наилучший результат покажет другая модель